In [6]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, VGAE
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

import pandas as pd
import json
from pathlib import Path

# --- 경로 설정 ---
ROOT_DIR = Path.cwd() 
DATA_DIR = ROOT_DIR / "BIgcontest_Data"
CROLLING_DIR = ROOT_DIR / "Crolling"
EDA_DIR = ROOT_DIR / "EDA"
OUTPUT_DIR = ROOT_DIR / "Model"

In [7]:
# ==============================================================================
# 1. 그래프 데이터 (노드, 엣지) 불러오기
# ==============================================================================
try:
    # 노드(가게)의 특징 데이터
    df_nodes = pd.read_csv(EDA_DIR/"nodes_with_coords.csv")
    # 엣지(가게 간 연결) 데이터
    df_edges = pd.read_csv(EDA_DIR/"edge_list.csv")
    print("✅ Step 1: 노드 및 엣지 데이터 로딩 성공!")
except FileNotFoundError:
    print("❌ 오류: 'nodes_with_coords.csv' 또는 'edge_list.csv' 파일을 찾을 수 없습니다.")
    exit()

✅ Step 1: 노드 및 엣지 데이터 로딩 성공!


In [9]:

# ==============================================================================
# 2. 모델 학습을 위한 데이터 전처리
# ==============================================================================
# 2-1. 노드(가게) ID를 0부터 시작하는 정수 인덱스로 매핑
# 딥러닝 모델은 문자열 ID 대신 숫자 인덱스를 사용합니다.
node_ids = df_nodes['ENCODED_MCT'].unique()
node_id_to_idx = {node_id: i for i, node_id in enumerate(node_ids)}

df_nodes['node_idx'] = df_nodes['ENCODED_MCT'].map(node_id_to_idx)

# 2-2. 엣지 리스트를 숫자 인덱스로 변환
df_edges['source_idx'] = df_edges['source'].map(node_id_to_idx)
df_edges['target_idx'] = df_edges['target'].map(node_id_to_idx)

# 유효한 엣지(source, target 모두 매핑된 경우)만 필터링
df_edges_clean = df_edges.dropna(subset=['source_idx', 'target_idx'])

# PyG가 사용하는 엣지 인덱스 형태로 변환 (2, num_edges)
edge_index = torch.tensor(df_edges_clean[['source_idx', 'target_idx']].values, dtype=torch.long).t().contiguous()

# 2-3. 노드 특징(Node Features) 스케일링
# 모델이 안정적으로 학습하도록 숫자형 특징들의 단위를 맞춰줍니다 (평균 0, 분산 1).
feature_cols = df_nodes.select_dtypes(include=np.number).columns.drop(['node_idx', 'latitude', 'longitude'])
scaler = StandardScaler()
df_nodes[feature_cols] = scaler.fit_transform(df_nodes[feature_cols])

# 특징 데이터를 PyTorch 텐서로 변환
node_features = torch.tensor(df_nodes[feature_cols].values, dtype=torch.float)

print("✅ Step 2: 데이터 전처리 및 PyTorch 텐서 변환 완료!")

✅ Step 2: 데이터 전처리 및 PyTorch 텐서 변환 완료!


In [10]:
# ==============================================================================
# 3. PyTorch Geometric (PyG) 그래프 데이터 객체 생성
# ==============================================================================
data = Data(x=node_features, edge_index=edge_index)
print("✅ Step 3: PyG 그래프 데이터 객체 생성 완료!")
print(data)

# ==============================================================================
# 4. VGAE 모델 정의
# ==============================================================================
# VGAE의 인코더 부분 정의
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # 두 개의 그래프 컨볼루션 레이어(GCN)를 사용
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        # mu(평균)와 logStd(로그 표준편차)를 위한 두 번째 GCN 레이어
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

# 모델 파라미터 설정
in_channels = data.num_node_features
out_channels = 16 

# ✅ 1. 사용할 장치(device) 설정: GPU가 있으면 cuda, 없으면 cpu로 자동 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Step 3.5: 연산에 사용할 장치 확인 --- {device}")


# VGAE 모델 생성
model = VGAE(VariationalGCNEncoder(in_channels, out_channels))

# ✅ 2. 모델과 데이터를 설정된 장치(GPU)로 이동
model.to(device)
data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("✅ Step 4: VGAE 모델 정의 및 GPU 설정 완료!")

✅ Step 3: PyG 그래프 데이터 객체 생성 완료!
Data(x=[4185, 30], edge_index=[2, 72426])
✅ Step 3.5: 연산에 사용할 장치 확인 --- cuda
✅ Step 4: VGAE 모델 정의 및 GPU 설정 완료!


In [11]:
# ==============================================================================
# 5. 모델 학습
# ==============================================================================
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(data.x, data.edge_index) # data가 GPU에 있으므로 이 연산도 GPU에서 수행됨
    loss = model.recon_loss(z, data.edge_index) + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)

print("\n--- Step 5: 모델 학습 시작 (Epoch 300) ---")
for epoch in range(1, 301):
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

print("\n✅ Step 5: 모델 학습 완료!")


--- Step 5: 모델 학습 시작 (Epoch 300) ---
Epoch: 010, Loss: 1.3250
Epoch: 020, Loss: 1.0096
Epoch: 030, Loss: 0.9288
Epoch: 040, Loss: 0.8940
Epoch: 050, Loss: 0.8787
Epoch: 060, Loss: 0.8702
Epoch: 070, Loss: 0.8590
Epoch: 080, Loss: 0.8537
Epoch: 090, Loss: 0.8504
Epoch: 100, Loss: 0.8448
Epoch: 110, Loss: 0.8427
Epoch: 120, Loss: 0.8401
Epoch: 130, Loss: 0.8362
Epoch: 140, Loss: 0.8353
Epoch: 150, Loss: 0.8344
Epoch: 160, Loss: 0.8345
Epoch: 170, Loss: 0.8301
Epoch: 180, Loss: 0.8293
Epoch: 190, Loss: 0.8289
Epoch: 200, Loss: 0.8243
Epoch: 210, Loss: 0.8248
Epoch: 220, Loss: 0.8230
Epoch: 230, Loss: 0.8247
Epoch: 240, Loss: 0.8241
Epoch: 250, Loss: 0.8225
Epoch: 260, Loss: 0.8221
Epoch: 270, Loss: 0.8233
Epoch: 280, Loss: 0.8192
Epoch: 290, Loss: 0.8225
Epoch: 300, Loss: 0.8197

✅ Step 5: 모델 학습 완료!


In [15]:
# ==============================================================================
# 6. 학습된 임베딩(잠재 벡터) 추출 및 저장
# ==============================================================================
# 학습된 모델을 사용해 최종 잠재 벡터(임베딩)를 추출
model.eval()
with torch.no_grad():
    z = model.encode(data.x, data.edge_index)

# 결과를 DataFrame으로 변환
df_embeddings = pd.DataFrame(z.cpu().numpy(), columns=[f'latent_{i}' for i in range(out_channels)])
df_embeddings['ENCODED_MCT'] = df_nodes['ENCODED_MCT']

print("\n--- Step 6: 학습된 잠재 벡터(임베딩) 추출 완료 ---")
print(df_embeddings.head())


# 다음 분석 단계에서 사용할 수 있도록 파일로 저장
home_dir = Path.home()
output_path = home_dir / "Bigcontest"/"Model"/"node_embeddings.csv"
df_embeddings.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n🎉 모든 작업 완료! 각 가게의 최종 잠재 벡터가 '{output_path}' 파일로 저장되었습니다.")


--- Step 6: 학습된 잠재 벡터(임베딩) 추출 완료 ---
   latent_0  latent_1  latent_2  latent_3  latent_4  latent_5  latent_6  \
0  0.239112 -0.167312 -0.204302 -0.318421 -0.100513 -0.033224  0.034737   
1 -0.062536  0.382447  0.197181 -0.101919  0.211323 -0.129040  0.160626   
2 -0.950546  0.232531 -0.143638  0.059959 -0.009538 -0.404986  0.816620   
3  0.087379 -0.071913  0.074773  0.696067  0.697613  0.098010  0.263093   
4 -0.041692  0.331113  0.023690  0.166006  0.281873  0.143643  0.323818   

   latent_7  latent_8  latent_9  latent_10  latent_11  latent_12  latent_13  \
0  0.641907 -0.102567  1.073260   0.090179   0.557739   0.418223   0.249325   
1  0.243525 -0.171747 -0.195750   0.067225  -0.261589   0.085779   0.609086   
2  0.300426  0.231899 -0.006807   0.107244   0.370142  -0.268149   0.472236   
3  0.073178  0.080390 -0.094903  -0.252551  -0.085186  -0.214601  -0.467186   
4 -0.277812 -0.114077 -0.282480  -0.641633   0.240527  -0.594519  -0.462066   

   latent_14  latent_15 ENCODED_MCT 